**Audio timstamping using WhisperX**

In [ ]:
! pip install git+https://github.com/m-bain/whisperx.git
!pip install google-cloud-vision PyMuPdf
# from google.colab import files
# uploaded = files.upload()
!pip install pydub
# !whisperx /content/ttf_ash_s24_masteraudio.mp3 --model medium.en --output_dir . --align_model WAV2VEC2_ASR_LARGE_LV60K_960H --highlight_words True

  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-hbnhb0v0
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-hbnhb0v0
  Resolved https://github.com/m-bain/whisperx.git to commit 8540ff5985fceee764acbed94f656063d7f56540
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/SYSTRAN/faster-whisper.git (to revision 0.10.0) to /tmp/pip-install-mnlv4jnn/faster-whisper_5e2a31e9e46a4b1c95e02eb1bde04a17
  Running command git clone --filter=blob:none --quiet https://github.com/SYSTRAN/faster-whisper.git /tmp/pip-install-mnlv4jnn/faster-whisper_5e2a31e9e46a4b1c95e02eb1bde04a17
  Running command git checkout -q e1a218fab1ab02d637b79565995bf1a9c4c83a09
  Resolved https://github.com/SYSTRAN/faster-whisper.git to commit e1a218fab1ab02d637b79565995bf1a9c4c83a09
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.6/208.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.1/442.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 28.2 MB/s eta 0:00:00


Before running the below cell add path to the adio uploaded above.
After running this cell the json and srt files will be generated

In [ ]:
import whisperx
import gc
from pydub import AudioSegment
import pandas as pd
import os
from PIL import Image
from google.cloud import vision
import json
import pandas as pd
import cv2
import fitz


device = "cuda"
batch_size = 16 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)
# 1. Transcribe with original whisper (batched)
model = whisperx.load_model("large-v2", device, compute_type=compute_type)

# save model to local path (optional)
# model_dir = "/path/"
# model = whisperx.load_model("large-v2", device, compute_type=compute_type, download_root=model_dir)
from google.colab import drive
drive.mount('/content/drive')


/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.10/dist-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

No language specified, language will be first be detected for each audio file (increases inference time).


100%|█████████████████████████████████████| 16.9M/16.9M [00:01<00:00, 15.1MiB/s]
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.1.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.0. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.0+cu121. Bad things might happen unless you revert torch to 1.x.
Mounted at /content/drive


In [ ]:

def convert_wav_to_mp3(input_file, output_file):
    sound = AudioSegment.from_wav(input_file)
    sound.export(output_file, format="mp3")

if __name__ == "__main__":
    input_wav_file = "/content/drive/MyDrive/ttf_ash_s24_masteraudio.wav"  # Replace with the path to your WAV file
    output_mp3_file = "/content/drive/MyDrive/ttf_ash_s24_masteraudio.mp3"  # Replace with the desired output path and file name

    convert_wav_to_mp3(input_wav_file, output_mp3_file)

audio_file = "/content/drive/MyDrive/ttf_ash_s24_masteraudio.wav"

audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)
print(result["segments"]) # before alignment

# delete model if low on GPU resources
# import gc; gc.collect(); torch.cuda.empty_cache(); del model


Detected language: en (0.99) in first 30s of audio...
[{'text': " Ash's Market Day Mayhem Written by Steve Fox Illustrated by Elena Dalaglio", 'start': 4.514, 'end': 14.241}, {'text': " Hey, down here. Bet you've never noticed us before, huh? We are the tiny folk. You know all the stuff you drop around the house or that falls out of your backpack? The pencils, buttons, paperclips, rubber bands, and hair ties you lose? We turn those things into our houses, tools, and clothes.", 'start': 20.538, 'end': 42.483}, {'text': ' Us tiny folk live in your gardens and yards, just out of sight. We grow itty-bitty plants, make friends with bugs and little animals, and enjoy our tiny lives on the edge of nature. But sometimes, tiny folk go on big adventures. When we help each other, no challenge is too large. Pawn? Pawn?', 'start': 44.326, 'end': 73.951}, {'text': " Pawn! Didn't you hear me shouting? Course I did, Ash. I heard you running too, so I just waited here. What's the hullabaloo? The big ti

In [ ]:
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

print(result["segments"]) # after alignment



Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth
100%|██████████| 360M/360M [00:01<00:00, 252MB/s]


[{'start': 4.674, 'end': 13.941, 'text': " Ash's Market Day Mayhem Written by Steve Fox Illustrated by Elena Dalaglio", 'words': [{'word': "Ash's", 'start': 4.674, 'end': 5.114, 'score': 0.759}, {'word': 'Market', 'start': 5.275, 'end': 5.695, 'score': 0.951}, {'word': 'Day', 'start': 5.775, 'end': 6.075, 'score': 0.828}, {'word': 'Mayhem', 'start': 6.195, 'end': 6.796, 'score': 0.895}, {'word': 'Written', 'start': 8.337, 'end': 8.697, 'score': 0.764}, {'word': 'by', 'start': 8.937, 'end': 9.137, 'score': 0.832}, {'word': 'Steve', 'start': 9.237, 'end': 9.578, 'score': 0.857}, {'word': 'Fox', 'start': 9.678, 'end': 10.078, 'score': 0.998}, {'word': 'Illustrated', 'start': 11.519, 'end': 12.16, 'score': 0.764}, {'word': 'by', 'start': 12.52, 'end': 12.7, 'score': 0.798}, {'word': 'Elena', 'start': 12.86, 'end': 13.24, 'score': 0.777}, {'word': 'Dalaglio', 'start': 13.34, 'end': 13.941, 'score': 0.896}]}, {'start': 20.578, 'end': 21.958, 'text': ' Hey, down here.', 'words': [{'word': 'He

In [ ]:
word=[]
start_time=[]
end_time=[]
for j in range(0,len(result["segments"])):
  for i in range(0,len(result["segments"][j]['words'])):
    try:
      word.append(result["segments"][j]['words'][i]['word'])
      start_time.append(result["segments"][j]['words'][i]['start'])
      end_time.append(result["segments"][j]['words'][i]['end'])

    except:
      #print(result["segments"][j]['words'][i]['word'])
      #word.append(result["segments"][j]['words'][i]['word'])
      start_time.append((result["segments"][j]['words'][i-1]['end'])+.005)
      end_time.append((result["segments"][j]['words'][i+1]['start'])-.005)

data = { 'start_time': start_time, 'end_time': end_time,'word': word}

# Creating a DataFrame
df = pd.DataFrame(data)
df.to_csv('output.txt', sep='\t', index=False)

sentence=[]
start_time=[]
end_time=[]
for i in range(0,len(result["segments"])):
  sentence.append(result["segments"][i]['text'])
  start_time.append(result["segments"][i]['start'])
  end_time.append(result["segments"][i]['end'])

data_se = { 'start_time': start_time, 'end_time': end_time,'sentence': sentence}
df_se = pd.DataFrame(data_se)

In [ ]:
df['end_time_shifted']=df['end_time'].shift(1)
df['diff']=df['start_time']-df['end_time_shifted']


# Assuming your DataFrame is named df

# Define a function to calculate diff_30% based on the given logic
def calculate_diff_30(diff):
    if diff < 0.1:
        return diff * 0.47
    elif 0.1 <= diff < 0.2:
        return diff * 0.42
    elif 0.2 <= diff < 0.5:
        return diff * 0.35
    elif 0.5 <= diff < 1.0:
        return diff * 0.2
    elif 1.0 <= diff < 5.0:
        return diff * 0.06
    elif 5.0 <= diff < 10.0:
        return diff * 0.04
    elif 10.0 <= diff < 50.0:
        return diff * 0.02
    else:
        return diff * 0.01

df['diff_30%'] = df['diff'].apply(calculate_diff_30)
df['diff_30%_shifted']=df['diff_30%'].shift(-1)
df['diff_30%_shifted'].fillna(0)

df['diff_30%']=df['diff_30%'].fillna(0)
df['new_start_time']=df['start_time']-df['diff_30%']

df['new_end_time']=df['end_time']+(df['diff_30%_shifted'])

df['new_start_time'].iloc[0]=df['start_time'].iloc[0]-df['start_time'].iloc[0]*.05

last=df.shape[0]
df['new_end_time'].iloc[last-1]=df['end_time'].iloc[last-1]+df['end_time'].iloc[last-1]*.05

df['len_words'] = df['word'].apply(lambda x: len(str(x)))
df['new_end_time_shifted']=df['new_end_time'].shift(1)
df['diff2']=df['new_start_time']-df['new_end_time_shifted']

df['final_start_time'] = df.apply(lambda row: row['new_start_time'] - (0.1 * row['diff2']) if row['diff2'] > 1 and len(row['word']) >= 6 else (row['new_start_time'] - (0.2 * row['diff2']) if len(row['word']) >= 6 else row['new_start_time']), axis=1)

df[['word','final_start_time','new_end_time']]

# df['diff_70%']=(df['diff']*.75)

# df['diff_70%'].fillna(0)

# df['new_start_time']=df['start_time']-df['diff_70%']+.0005
# df['new_start_time'].iloc[0]=df['start_time'].iloc[0]

# df['new_end_time'].iloc[last-1]=df['end_time'].iloc[last-1]

,word,final_start_time,new_end_time
0,Ash's,4.440300,5.18162
1,Market,5.202228,5.73260
2,Day,5.737400,6.12540
3,Mayhem,6.140760,6.88846
4,Written,8.108932,8.78100
...,...,...,...
857,it,519.589800,519.71680
858,all,519.739200,520.02520
859,up?,520.050800,520.29886
860,The,521.778140,522.26320


In [ ]:
df[['word','final_start_time','new_end_time']].head(50)

,word,final_start_time,new_end_time
0,Ash's,4.440300,5.18162
1,Market,5.202228,5.73260
2,Day,5.737400,6.12540
3,Mayhem,6.140760,6.88846
4,Written,8.108932,8.78100
5,by,8.853000,9.18400
6,Steve,9.190000,9.62000
7,Fox,9.636000,10.16446
8,Illustrated,11.305732,12.28600
9,by,12.394000,12.76720


In [ ]:
df[['word','final_start_time','new_end_time']].to_csv("AUDIO_OUTPUT_28dec.csv",index=False)

In [ ]:
df=df_se

In [ ]:
df['end_time_shifted']=df['end_time'].shift(1)
df['diff']=df['start_time']-df['end_time_shifted']
df['diff_30%']=(df['diff']*.25)
df['diff_70%']=(df['diff']*.75)
df['diff_30%_shifted']=df['diff_30%'].shift(-1)
df['diff_70%'].fillna(0)
df['diff_30%_shifted'].fillna(0)
df['new_end_time']=df['end_time']+df['diff_30%_shifted']
df['new_start_time']=df['start_time']-df['diff_70%']+.0005
df['new_start_time'].iloc[0]=df['start_time'].iloc[0]
last=df.shape[0]
df['new_end_time'].iloc[last-1]=df['end_time'].iloc[last-1]

In [ ]:
df[['new_start_time','new_end_time','sentence']]

,start_time,end_time,sentence,end_time_shifted,diff,diff_30%,diff_70%,diff_30%_shifted,new_end_time,new_start_time
0,4.674,21.967,Ash's Market Day Mayhem Written by Steve Fox ...,NaN,NaN,NaN,NaN,0.22000,22.18700,4.67400
1,22.847,24.889,"Bet you've never noticed us before, huh?",21.967,0.880,0.22000,0.66000,0.16500,25.05400,22.18750
2,25.549,27.011,We are the tiny folk.,24.889,0.660,0.16500,0.49500,0.25400,27.26500,25.05450
3,28.027,32.392,You know all the stuff you drop around the ho...,27.011,1.016,0.25400,0.76200,0.12000,32.51200,27.26550
4,32.872,37.578,"The pencils, buttons, paperclips, rubber bands...",32.392,0.480,0.12000,0.36000,0.20000,37.77800,32.51250
...,...,...,...,...,...,...,...,...,...,...
119,500.327,505.330,"You, me, our animal pals, and all the tiny fol...",499.286,1.041,0.26025,0.78075,0.42525,505.75525,499.54675
120,507.031,508.772,It's not the market I had expected.,505.330,1.701,0.42525,1.27575,0.25000,509.02200,505.75575
121,509.772,510.753,It's even better.,508.772,1.000,0.25000,0.75000,0.36025,511.11325,509.02250
122,512.194,520.198,"Now, the only question is, when the party's ov...",510.753,1.441,0.36025,1.08075,0.42025,520.61825,511.11375


In [ ]:
# Code For Recreating

In [ ]:
import shutil

folder_paths = [
    '/content/image-to-livetext-1/data/images',
    '/content/image-to-livetext-1/data/jsons'

]
for folder_path in folder_paths:
    try:
        shutil.rmtree(folder_path)
        print(f"Deleted folder: {folder_path}")
    except Exception as e:
        print(f"Error deleting folder {folder_path}: {e}")

In [ ]:
# Take all Crednetial
!unzip -q "/content/drive/MyDrive/image-to-livetext-1 (2).zip"

In [ ]:
# Set up credentials and environment
credentials_path = r'/content/image-to-livetext-1/credentials.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
client = vision.ImageAnnotatorClient()

def detect_text(image_path):
    # Reads the image file
    with open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    # Performs text detection on the image file
    response = client.text_detection(image=image)

    fullTextAnnotation = response.full_text_annotation
    json_data = vision.TextAnnotation.to_json(fullTextAnnotation)

    return json_data

def get_image_size(image_path):
    with Image.open(image_path) as img:
        return img.size

def process_images_in_folder(folder_path,json_folder):
    # Create a folder for JSON files
    parent_folder = os.path.dirname(folder_path)
    if json_folder=='':
      json_folder = os.path.join(parent_folder, 'jsons')
    html_folder = os.path.join(parent_folder, 'htmls')
    os.makedirs(json_folder, exist_ok=True)
    os.makedirs(html_folder, exist_ok=True)

    # Iterate over the images in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)

            # Check if a corresponding JSON file already exists
            response_file = os.path.splitext(filename)[0] + '.json'
            response_path = os.path.join(json_folder, response_file)
            json_exists = os.path.exists(response_path)

            if not json_exists:
                text_annotation = detect_text(image_path)
                # print(text_annotation)

                with open(response_path, 'w', encoding='utf-8') as fle:
                    fle.write(text_annotation)

            else:
                # Read the contents of the existing JSON file
                with open(response_path, 'r') as fle:
                    text_annotation = fle.read() # json.load(fle


def extract_text(element):
        if "text" in element:
            return element["text"]
        elif "symbols" in element:
            return "".join([symbol["text"] for symbol in element["symbols"]])
        elif "words" in element:
            return " ".join([extract_text(word) for word in element["words"]])
        elif "paragraphs" in element:
            return "\n".join([extract_text(paragraph) for paragraph in element["paragraphs"]])
        else:
            return ""


def get_text_coord_from_json(file_path):

    #file_path = f"/content/image-to-livetext-1/data/jsons/masked-image.json"

    text_list = []
    left_list = []
    width_list = []
    height_list = []
    top_list = []

    with open(file_path, "r", encoding="utf-8") as json_file:
        data = json.load(json_file)

            # Extract text, paragraphs, and their bounding boxes
        text = data["text"]
        paragraphs = []

        for page in data["pages"]:
                for block in page["blocks"]:
                    if "paragraphs" in block:
                        for paragraph in block["paragraphs"]:
                            paragraph_text = extract_text(paragraph)
                            paragraph_bbox = paragraph["boundingBox"]["vertices"]

                            # Extract coordinates
                            left, top = paragraph_bbox[0]['x'], paragraph_bbox[0]['y']
                            width = paragraph_bbox[2]['x'] - left
                            height = paragraph_bbox[2]['y'] - top

                            # Append data to the lists
                            #file_list.append(filename)
                            text_list.append(paragraph_text)
                            left_list.append(left)
                            top_list.append(top)
                            width_list.append(width)
                            height_list.append(height)

    # Create a DataFrame
    item_df = pd.DataFrame({
        "text": text_list,
        "Left": left_list,
        "Top": top_list,
        "Width": width_list,
        "Height": height_list
    })

    return item_df



In [ ]:
output_dir = '/content/pdf-image'
os.makedirs(output_dir, exist_ok=True)    # Create the directory if it doesn't exist
pdf_path='/content/drive/MyDrive/ttf_ash_s24.pdf'
# Open the PDF file
pdf_document = fitz.open(pdf_path)

# Iterate through the pages and save them as images
for page_number in range(pdf_document.page_count):
    page = pdf_document.load_page(page_number)
    image = page.get_pixmap()
    image_width = image.width
    image_height = image.height
    pil_image = Image.frombytes("RGB", [image.width, image.height], image.samples)
    image_file_path = os.path.join(output_dir, f'page_{page_number}.png')
    pil_image.save(image_file_path)

In [ ]:
import glob

png_files = glob.glob(f"{output_dir}/*.png")
count_png_files = len(png_files)



In [ ]:
count_png_files

In [ ]:
data=pd.DataFrame()
for page_number in range(0,count_png_files):
  image = cv2.imread(f'/content/pdf-image/page_{page_number}.png')  # Update the image path
  _, img_bytes = cv2.imencode('.jpg', image)
  output_path = f'/content/image-to-livetext-1/data/images/page_{page_number}.png'
  with open(output_path, 'wb') as f:
      f.write(img_bytes.tobytes())

  folder_path = '/content/image-to-livetext-1/data/images'
  process_images_in_folder(folder_path,"")
  json_path =  f"/content/image-to-livetext-1/data/jsons/page_{page_number}.json"
  para_df = get_text_coord_from_json(json_path)

  threshold = 500

  # Convert the 'text' column to strings
  para_df['text'] = para_df['text'].astype(str)
  single_char_text = para_df[para_df['text'].str.len() == 1]

  # Check if there are any single-character entries
  if not single_char_text.empty:
      # Find the single character with the highest 'Top' value
      highest_top_single_char_text = single_char_text[single_char_text['Top'] == single_char_text['Top'].max()]

      # Remove all single-character text entries except the one with the highest 'Top'
      para_df = para_df[~((para_df['text'].str.len() == 1) & (para_df['Top'] != highest_top_single_char_text.iloc[0]['Top']) )]

  # Filter out entries based on the threshold
  para_df = para_df[abs(para_df['Width'] * para_df['Height']) >= threshold]
  para_df.reset_index(drop=True, inplace=True)
  modified_para_df = para_df.copy()
  min_height = 45
  max_text_length = 26
  min_text_length = 3
  def assign_category(row):
      if (row['Height'] > min_height and min_text_length < len(row['text']) <= max_text_length):
          return 'title'
      else:
          return 'text'
  modified_para_df['category_name'] = modified_para_df.apply(assign_category, axis=1)

  modified_para_df['page']=page_number+1
  data=pd.concat([data,modified_para_df],axis=0)


In [ ]:
df.head()

In [ ]:
para=list(data['text'])

In [ ]:
para

In [ ]:
# Initialize variables
current_paragraph = 0
paragraphs = []
for item in list(df['word'])[0:10]:
  current_word = np.char.lower(item)
  # Check if the current word is in the current paragraph
  temp=[]
  temp_para=[]
  #for i in range(0, len(para)):
  for i in range(0,5):
    parhs = [word.lower() for word in para[i].split()]
    print(current_word,parhs)
    if current_word in parhs:
      temp.append(current_word)
      temp_para.append(para[i])

In [ ]:
df.head(20)

In [ ]:
for i in range(0,5):
    parhs = [word.lower() for word in para[i].split()]
    print(parhs)
    for j in parhs:
      df1=df[df['word']==j]
      print(j)
      print(df1.head(1))


In [ ]:
# Given data
word_data = df

# Initialize variables
current_paragraph = 0
paragraphs = []

# Iterate through word data to match with paragraphs
for item in word_data:
    current_word = item['word']

    # Check if the current word is in the current paragraph
    if current_word in para[current_paragraph]:
        paragraphs.append(current_word)
    else:
        # Move to the next paragraph
        current_paragraph += 1
        # Check if the current word is still in the current paragraph after moving
        if current_word in para[current_paragraph]:
            paragraphs.append(current_word)
        else:
            # The current word spans across multiple paragraphs, split it
            current_word_parts = current_word.split()
            paragraphs.extend(current_word_parts)

# Combine words to form a logical paragraph
logical_paragraph = ' '.join(paragraphs)

# Print the logical paragraph
print("Logical Paragraph:")
print(logical_paragraph)



In [ ]:
import numpy as np

def word_exists(word, word_list):
    for element in word_list:
        if word in np.char.lower(element):
            return True
    return False






In [ ]:
for i in
word_exists(np.char.lower(df['word'].iloc[0]), np.char.lower(data['text'].iloc[2].split()))

In [ ]:
def calculate_paragraph_timestamp(row):
    words = row['text'].split()
    start_times = df[df['word'].isin(words)]['start_time']
    end_times = df[df['word'].isin(words)]['end_time']
    return start_times.min(), end_times.max()

# Apply the function to each row in the 'data' dataframe
data[['paragraph_start_time', 'paragraph_end_time']] = data.apply(calculate_paragraph_timestamp, axis=1, result_type='expand')



In [ ]:
data.head(10)

In [1]:
#### GCP code

In [13]:
!pip install google-cloud-speech
!pip install pydub
# --------------------------------Google Speech to Text API--------------------------------

import io
import os
from google.cloud import speech_v1p1beta1 as speech
from pydub import AudioSegment

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
def stereo_to_mono(audio_file_path):
    audio = AudioSegment.from_wav(audio_file_path)
    audio = audio.set_channels(1)  # Convert to mono (single channel)
    temp_mono_file = "temp_mono_audio.wav"
    audio.export(temp_mono_file, format="wav")
    return temp_mono_file

In [16]:
import os
os.chdir(r'/content/drive/MyDrive')

In [19]:
# Replace "page0001.wav" with the path to your audio file
path = r"/content/drive/MyDrive/ttf_ash_s24_masteraudio.wav"
audio_file_path = stereo_to_mono(path)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credentials.json"
client = speech.SpeechClient()
with io.open(audio_file_path, "rb") as audio_file:
    content = audio_file.read()
audio = speech.RecognitionAudio(content=content)
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=48000,
    language_code="en-US",
    enable_word_time_offsets=True  # Enable word-level timestamps
)
audio_file_name = os.path.splitext(os.path.basename(path))[0]
response = client.recognize(config=config, audio=audio)

InvalidArgument: ignored

In [24]:
# Replace "page0001.wav" with the path to your audio file
path = r"/content/drive/MyDrive/ttf_ash_s24_masteraudio.wav"
audio_file_path = stereo_to_mono(path)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credentials.json"
client = speech.SpeechClient()

def generate_audio_chunks(audio_path, chunk_size=524288):  # 512 KB chunk size
    with io.open(audio_path, "rb") as audio_file:
        while True:
            chunk = audio_file.read(chunk_size)
            if not chunk:
                break
            yield chunk

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=48000,
    language_code="en-US",
    enable_word_time_offsets=True  # Enable word-level timestamps
)

audio_file_name = os.path.splitext(os.path.basename(path))[0]

streaming_config = speech.StreamingRecognitionConfig(
    config=config, interim_results=True
)

requests = (
    speech.StreamingRecognizeRequest(audio_content=chunk) for chunk in generate_audio_chunks(audio_file_path)
)

streaming_api_response = client.streaming_recognize(config=streaming_config, requests=requests)

for response in streaming_api_response:
    for result in response.results:
        print("Transcript: {}".format(result.alternatives[0].transcript))

Transcript: ashes
Transcript: ashes Market
Transcript: ashes
Transcript:  Market
Transcript: ashes Market
Transcript: ashes Market
Transcript:  Day May
Transcript: ashes Market
Transcript:  de Mayo
Transcript: ashes Market
Transcript:  day may have
Transcript: ashes Market
Transcript:  Day Mayhem
Transcript: ashes Market Day
Transcript:  Mayhem
Transcript: ashes Market Day Mayhem
Transcript: ashes Market Day Mayhem
Transcript:  bring
Transcript:  Britt
Transcript:  Brittany
Transcript:  written by
Transcript:  Brittany B
Transcript:  Brittany b e
Transcript:  written by Steve
Transcript:  Britney Spears
Transcript:  written by Steve
Transcript:  written by Stephen
Transcript:  written by theme song
Transcript:  written by Stephen
Transcript:  written by theme song
Transcript:  Ritz spicy sauce
Transcript:  Brittany b c songs
Transcript:  Ritz spicy Fox
Transcript:  Brittany b c songs
Transcript:  Brittany b e Fox
Transcript:  it was
Transcript:  illustrate
Transcript:  Illustrated
Tran

In [28]:
# old
def create_transcript_file(audio_file_path):
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credentials.json"
    client = speech.SpeechClient()
    with io.open(audio_file_path, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=48000,
        language_code="en-US",
        enable_word_time_offsets=True  # Enable word-level timestamps
    )

    audio_file_name = os.path.splitext(os.path.basename(path))[0]
    response = client.recognize(config=config, audio=audio)
    print(response)

    output_file = f"{audio_file_name}_transcript.txt"

    with open(output_file, "w") as file:
        # Process and write the results with word-level timestamps
        for result in response.results:
            if result.alternatives:
                alternative = result.alternatives[0]

                if alternative.words:
                    for word_info in alternative.words:
                        word = word_info.word
                        start_time = word_info.start_time
                        end_time = word_info.end_time

                        file.write(
                            f"{start_time.seconds:.3f}\t"
                            f"{end_time.seconds:.3f}\t{word}\n"
                        )
                else:
                    file.write("No words found in the response.\n")
            else:
                file.write("No alternatives found in the response.\n")

    return output_file

